In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.metrics import (
    roc_curve, auc,
    precision_recall_curve, average_precision_score,
    confusion_matrix, f1_score, roc_auc_score
)
import glob

File Creation

In [ ]:
os.makedirs("ep_robust_viz", exist_ok=True)
os.makedirs("ep_robust_log", exist_ok=True)

Plotting Functions

In [ ]:
def plot_avg_history(epochs, avg_metrics, std_metrics, epsilons):
    plt.figure(figsize=(12, 6))

    # Loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, avg_metrics['loss'], label='Train Loss')
    plt.fill_between(epochs,
                     np.array(avg_metrics['loss']) - np.array(std_metrics['loss']),
                     np.array(avg_metrics['loss']) + np.array(std_metrics['loss']),
                     alpha=0.2)
    plt.plot(epochs, avg_metrics['val_loss'], label='Val Loss')
    plt.fill_between(epochs,
                     np.array(avg_metrics['val_loss']) - np.array(std_metrics['val_loss']),
                     np.array(avg_metrics['val_loss']) + np.array(std_metrics['val_loss']),
                     alpha=0.2)
    plt.title("Loss over Epochs")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()

    # Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, avg_metrics['accuracy'], label='Train Accuracy')
    plt.fill_between(epochs,
                     np.array(avg_metrics['accuracy']) - np.array(std_metrics['accuracy']),
                     np.array(avg_metrics['accuracy']) + np.array(std_metrics['accuracy']),
                     alpha=0.2)
    plt.plot(epochs, avg_metrics['val_accuracy'], label='Val Accuracy')
    plt.fill_between(epochs,
                     np.array(avg_metrics['val_accuracy']) - np.array(std_metrics['val_accuracy']),
                     np.array(avg_metrics['val_accuracy']) + np.array(std_metrics['val_accuracy']),
                     alpha=0.2)
    plt.title("Accuracy over Epochs")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend()

    plt.tight_layout()
    plt.savefig(f"ep_robust_viz/{epsilons[idx]}/history_curves.png")
    plt.close()

In [ ]:
def plot_mean_se_with_baseline(mean_se_values, x_values, x_label, y_label, filename,
                                baseline, baseline_label="Baseline"):

    os.makedirs("robust_viz", exist_ok=True)

    means = np.array([m for m, se in mean_se_values])
    ses = np.array([se for m, se in mean_se_values])

    # Extend data with baseline
    baseline_mean, baseline_se = baseline
    means = np.append(means, baseline_mean)
    ses = np.append(ses, baseline_se)
    x_all = list(x_values) + [baseline_label]

    x_ticks = np.arange(len(x_all))

    plt.figure(figsize=(10, 5))
    plt.errorbar(x_ticks, means, yerr=ses, fmt='o-', capsize=5,
                 color='steelblue', ecolor='gray', elinewidth=2, marker='o')

    plt.xticks(ticks=x_ticks, labels=x_all, rotation=45)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(f"{y_label} across epsilons with SE and Baseline")
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f"robust_viz/{filename}")
    plt.close()

In [ ]:
def plot_confusion_matrix_with_se(conf_matrix, annotations, epsilons):
    plt.figure(figsize=(6, 5))
    sns.heatmap(conf_matrix, annot=annotations, fmt='', cmap="Blues", cbar=False, square=True,
                xticklabels=['Predicted 0', 'Predicted 1'],
                yticklabels=['Actual 0', 'Actual 1'])

    plt.title(f'Average Confusion Matrix with SE for epsilon {{epsilons[idx]}}')
    plt.xlabel('Prediction')
    plt.ylabel('Actual')
    plt.tight_layout()
    plt.savefig(f"ep_robust_viz/{epsilons[idx]}/avg_confusion_matrix.png")
    plt.close()

In [ ]:
def plot_metric_distribution(values, metric_name, filename, epsilons):
    mean, ci_lower, ci_upper = np_95ci(values)

    plt.figure(figsize=(8, 5))
    plt.hist(values, bins=15, color='skyblue', edgecolor='black', alpha=0.7)
    plt.axvline(mean, color='red', linestyle='--', label=f'Mean = {mean:.3f}')
    plt.axvline(ci_lower, color='green', linestyle=':', label=f'95% CI Lower = {ci_lower:.3f}')
    plt.axvline(ci_upper, color='green', linestyle=':', label=f'95% CI Upper = {ci_upper:.3f}')

    plt.title(f'{metric_name} Distribution with 95% CI')
    plt.xlabel(metric_name)
    plt.ylabel('Frequency')
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"ep_robust_viz/{epsilons[idx]}/{filename}")
    plt.close()

STATS FUNCTIONS

In [ ]:
def np_95ci(data):
    mean = np.mean(data)
    std = np.std(data, ddof=1)  # sample standard deviation
    se = std / np.sqrt(len(data))
    ci_lower = mean - 1.96 * se
    ci_upper = mean + 1.96 * se
    return mean, ci_lower, ci_upper

In [ ]:
def mean_se(values):
    values = np.array(values)
    return np.mean(values), np.std(values, ddof=1) / np.sqrt(len(values))

In [ ]:
def bootstrap(x_train, y_train):
    x_train = pd.DataFrame(x_train)
    y_train = pd.DataFrame(y_train)
    k = len(x_train)
    idx = np.random.choice(k, size = k,  replace = True)
    return x_train.iloc[idx], y_train.iloc[idx]

Preprocessing Functions

In [ ]:
def noisydata(data: pd.DataFrame, epsilon: float) -> pd.DataFrame:
    """
    Applies the Laplace mechanism to add noise to each feature column for differential privacy.
    Uses two-step mean + residual perturbation under a public bound of [0, 20], which covers the
    full range of log-transformed RNA-sequencing expression values in almost every practical dataset.

    Parameters:
            data (pd.DataFrame): input dataset.
            epsilon (float): privacy budget for differential privacy.

    Returns:
            pd.DataFrame: A DataFrame of the same shape as `data`, with Laplace noise added to each feature.
            The 'disease' column, if present, is left unchanged.
    """
    # Check if the target variable 'disease' is present in the data.
    if 'disease' in data.columns:
        X = data.drop(columns = ['disease'])
    else:
        X = data.copy()

    # Use public bounds [0, 20] for the features.
    a_public = pd.Series(0, index = X.columns)
    b_public = pd.Series(20, index = X.columns)

    # Clip the data to public [a, b].
    X_clipped = X.clip(lower = a_public, upper = b_public, axis = 1)

    # Split the privacy budget equally between the mean and residual perturbation.
    eps_mean = epsilon / 2
    eps_residual = epsilon / 2

    # Compute the true column means.
    n = len(X_clipped)
    mu = X_clipped.mean(axis = 0)

    # Privatize the means with Laplace ((U-L)/(n*eps_mean))
    sensitivity_mean = (b_public - a_public) / n
    scale_mean = sensitivity_mean / eps_mean
    noise_mean = np.random.laplace(loc = 0, scale = scale_mean.values, size = scale_mean.shape)
    mu_noisy = pd.Series(mu.values + noise_mean, index = X.columns)

    # Center on the noisy means.
    R = X_clipped.subtract(mu_noisy, axis = 1)

    # Compute the public residual bounds ((U-L)/2) = 10.
    D = (b_public - a_public) / 2

    # Clip the residuals to public bounds.
    R_clipped = R.clip(lower = -D, upper = D, axis = 1)

    # Privatize residuals with Laplace (2D / eps_residual).
    sensitivity_residual = 2 * D
    scale_residual = sensitivity_residual / eps_residual
    noise_residual = np.random.laplace(loc = 0 , scale = scale_residual.values, size = R_clipped.shape)
    R_noisy = pd.DataFrame(R_clipped.values + noise_residual, columns = X.columns, index = X.index)
    X_noisy = R_noisy.add(mu_noisy, axis = 1)
    noisydata = X_noisy

    # If 'disease' column is present, add it back to the noisy data.
    if 'disease' in data.columns:
        noisydata['disease'] = data['disease']

    # Return the noisy data.
    return noisydata

In [ ]:
def data_gen(epsilons, test_size =.2, random_state = 42):
    data_holder = []
    for idx, ep in enumerate(epsilons):
        noisy_data = noisydata(data, ep)

        X = noisy_data.drop(columns=['disease']).values
        y = noisy_data['disease'].values

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=test_size, random_state=random_state)
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        X_val = scaler.transform(X_val)

        data_holder.append((X_train, X_test, y_train, y_test, X_val, y_val))

    return data_holder

Preprocessing

In [ ]:
#TODO: You may need to fix file paths to whatever this is on your cluster
data = pd.read_csv('../data/endometriosis_dataset.csv')
epsilons = [.001, .01, .1, 1, 10]

In [ ]:
dataset = data_gen(epsilons, test_size =.2, random_state = 42)

Model Training Functions

In [ ]:
def model_train(data):

    for idx in range(len(data)):

        for i in range(50):

            X_boot, y_boot = bootstrap(data[idx][0])


            DP_model = Sequential([
            Dense(512, activation='relu', input_shape=(data[idx][0].shape[1],)),
            Dropout(0.5),
            Dense(128, activation='relu'),
            Dropout(0.5),
            Dense(1, activation='sigmoid')
        ])
            DP_model.compile(optimizer=Adam(learning_rate=1e-3),
                      loss='binary_crossentropy',
                      metrics=['accuracy'])

            csv_logger = tf.keras.callbacks.CSVLogger(f"ep_robust_log/{epsilons[idx]}/training_log_dp_{i}.csv", append=True)

            checkpoint = tf.keras.callbacks.ModelCheckpoint(
            filepath=f"models_dp_robust/{epsilons[idx]}/model_dp_{i}_{{epoch:02d}}.keras",
            save_weights_only=False,
            save_best_only=False,  # Save every epoch
            verbose=1
            )

            history_dp = DP_model.fit(
            X_boot,
            y_boot,
            epochs=50,
            batch_size=32,
            validation_data = (data[idx][4], data[idx][5]),
            callbacks = [csv_logger, checkpoint]
            )

Model Training

In [ ]:
model_train(dataset)

STATS Scripts

In [ ]:
f1_total = []
auc_total = []

for idx in range(len(epsilons)):
    # Step 1: Load all CSVs into a list of DataFrames
    csv_files = glob.glob(f"ep_robust_log/{epsilons[idx]}/training_log_dp_*.csv")
    histories = [pd.read_csv(f) for f in csv_files]

    # Step 2: Stack the metrics for each epoch
    metrics = ['loss', 'accuracy', 'val_loss', 'val_accuracy']
    avg_metrics = {m: [] for m in metrics}
    std_metrics = {m: [] for m in metrics}
    epochs = histories[0]['epoch']  # Assuming all runs have the same epoch range

    for epoch in epochs:
        for metric in metrics:
            values = [h.loc[epoch, metric] for h in histories]
            avg_metrics[metric].append(np.mean(values))
            std_metrics[metric].append(np.std(values))


    model_paths = glob.glob(f"models_dp_robust/{epsilons[idx]}/model_dp_*_50.keras")
    model_paths.sort()

    tp_list = []
    fp_list = []
    tn_list = []
    fn_list = []
    f1_scores = []
    auc_scores = []


    for model_path in model_paths:
        model = tf.keras.models.load_model(model_path)
        y_pred_prob = model.predict(X_test)
        y_pred = (y_pred_prob > 0.5).astype(int)

        f1 = f1_score(y_test, y_pred)
        auc = roc_auc_score(y_test, y_pred_prob)

        f1_scores.append(f1)
        auc_scores.append(auc)

        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

        tp_list.append(tp)
        fp_list.append(fp)
        tn_list.append(tn)
        fn_list.append(fn)

    # Compute mean and SE for each confusion matrix component
    tp_mean, tp_se = mean_se(tp_list)
    fp_mean, fp_se = mean_se(fp_list)
    tn_mean, tn_se = mean_se(tn_list)
    fn_mean, fn_se = mean_se(fn_list)

    # Construct the matrix and annotation array
    conf_matrix = np.array([[tn_mean, fp_mean],
                            [fn_mean, tp_mean]])

    annotations = np.array([[f"{tn_mean:.1f}\n±{tn_se:.1f}", f"{fp_mean:.1f}\n±{fp_se:.1f}"],
                            [f"{fn_mean:.1f}\n±{fn_se:.1f}", f"{tp_mean:.1f}\n±{tp_se:.1f}"]])

    f1_total.append(mean_se(f1_scores))
    auc_total.append(mean_se(auc_scores))

    plot_metric_distribution(f1_scores, "F1 Score", "f1_score_robust", epsilons)
    plot_metric_distribution(auc_scores, "AUC Score", "AUC_score_robust", epsilons)
    plot_confusion_matrix_with_se(conf_matrix, annotations, epsilons)
    plot_avg_history(epochs, avg_metrics, std_metrics, epsilons)

In [ ]:
f1_baseline = (0.9659922307568748, 0.002160392362343558)
auc_baseline = (0.896936507936508, 0.00458851871947309)

In [ ]:
plot_mean_se_with_baseline(f1_total, epsilons, "epsilons", "Average F1 Score", "avg_f1",
                                f1_baseline, baseline_label="baseline")

In [ ]:
plot_mean_se_with_baseline(auc_total, epsilons, "epsilons", "Average AUC Score", "avg_auc",
                                auc_baseline, baseline_label="baseline")